<a href="https://colab.research.google.com/github/alimoorreza/CS167-sp25-notes/blob/main/Day04_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS167: Day04
## k-Nearest-Neighbor

#### CS167: Machine Learning, Spring 2025


📜 [Syllabus](https://analytics.drake.edu/~reza/teaching/cs167_sp25/cs167_syllabus_sp25.pdf)

In [2]:
#run this cell if you're using Colab:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#import the data:
import pandas as pd

path2 = '/content/drive/MyDrive/cs167_sp25/datasets/irisData.csv'
iris= pd.read_csv(path2)
iris.head()


,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## 🚩 Notebook #1 Help

A few helpful functions for Notebook #1:
- `max()` will return the maximum value in a Series
- `idxmax()` will return the __index__ of the maximum value

## Quick Simple Stats Review:

- __mean__: the average, take the sum of elements, and divide by the number of total elements
- __median__: The middle number; found by ordering all data poits and picking out the one in the middle (or if there are two numbers in the middle, taking the mean of those two numbers).
- __mode__: The most frequent number; the number that occurs the highest number of times.

### [2,2,3,3,4,4,4]
Find the:
- mean:
- median:
- mode:

In [ ]:
import pandas as pd
numbers = [2,2,3,3,4,4,4]
ds = pd.Series(numbers)
print('mean: ', ds.mean())
print('median: ', ds.median())
print('mode: ', ds.mode())


mean:  3.142857142857143
median:  3.0
mode:  0    4
dtype: int64


## Machine Learning Variations

We are going to learn about a lot of different types of machine learning in CS167. Here are a few categories to look out for:
- __classifcation__: identify which category it goes in. Examples: Spam or ham? Eric or Tim? Fish, amphibian, reptile, bird, or mammal
- __regression__: real-valued labels. Examples: price of Bitcoin, tomorrow's temperature, etc.
- __supervised learning__: data has labels, goal is to predict the labels of new instance.
- __unsupervised learning__: data does not have a label, the goal is to analyze/cluster the examples.
- __other issues__: missing data, sequential data, outlier anomaly detetion, and many more.

Let's take a look at a couple rows of the Iris Dataset:

In [ ]:
iris.head(2)

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa


## 🚨 Terminology Alert 🚨
Each row in the table represents a __training example__, a previously-seen, known instance of the thing we are trying to model.

Each column in the table represents a __feature__, some attribute or variable that each training example has a value for.

__Target variable__: the 'feature' we will try to predict(e.g. species)--it's value is unkonwn for any new cases not in the training data.

__Predictor variables__: (or just predictors), the features that will be used to make predictions of the target variable. (e.g. `sepal length`, `petal length`, `sepal width`, `petal width`

Remember this question from Day01?

- 5.1 cm petal, 7.2 cm sepal

<div>
<img src="https://analytics.drake.edu/~reza/teaching/cs167_sp25/notes/images/day0_iris_q1.png?raw=1"/>
</div>


# Our First Machine Learning Model: k Nearest Neighbors
We're starting with a relatively simple, but foundational, machine learning model, the k-Nearest Neighbors (kNN) algorithm.

__k-Nearest-Neighbor Algorithm__: Predict the _most commonly appearing_ class among the __k__ closest training examples.

## 1-Nearest-Neighbor Algorithm

__k-Nearest-Neighbor Algorihm__: Precict the _most commonly appearing_ class among the __k__ closest training examples.
- in a 1-NN algorithm k=1

### What will a 1NN algorithm predict?


<div>
<img src="https://analytics.drake.edu/~reza/teaching/cs167_sp25/notes/images/day03_1NN_iris.png?raw=1" width=450/>
</div>

It will predict __Iris-versicolor__ because the nearest neighbor (shown by the black line) is Iris-versicolor.

## 3-Nearest-Neighbor Algorithm

> Wait... why did we skip 2-NN?

### What will a 3NN algorithm predict?

<div>
<img src="https://analytics.drake.edu/~reza/teaching/cs167_sp25/notes/images/day03_3NN_iris.png?raw=1" width=450/>
</div>

In a 3-Nearest Neighbor algorithm k=3. So, let's look at the 3 closest neighbor to our iris.
- The 3 closest neighbors are: `Iris-versicolor, Iris-virginica, Iris-virginica`
- a 3NN algorithm will predict the _most commonly appearing class_ among the k closest neighbors, so it will predict __Iris-virginica__.

## What do you mean by "closest"?
__k-Nearest-Neighbor Algorihm__: Precict the _most commonly appearing_ class among the __k__ closest training examples.

Alright... but how do we determine which training examples are the 'closest'?

__defining 'nearness':__ as the machine learning engineer, we get to choose how we define _close_.
- What ways can you think of to determine a distane between any two training examples?

# Distances:

## Euclidean Distance:

Example in 2D space: distance between (x1, y1) and (x2, y2) is:

## $\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2}$

## 2D Eucliden Distance Example:
Find the distance between (1,2) and (4,6):

$\sqrt{(1 - 4)^2 + (2 - 6)^2}$

$\sqrt{(-3)^2 + (-4)^2}$

$\sqrt{9 + 16}$

$\sqrt{25}$

$5$


## Euclidean Distance in n-Dimensional Space:

Suppose features of an example $x$ are denoted $\langle a_1(x), a_2(x), ... , a_n(x) \rangle$
- where $n$ is the number of features.

Then the Euclidean Distance:

## $d(x_i, x_j) = \sqrt{ \sum_{r=1}^{n} (a_r(x_i) - a_r(x_j))^2}$

## $d(x_i, x_j) = \sqrt{ \sum_{r=1}^{n} (a_r(x_i) - a_r(x_j))^2}$

Example: Suppose we find a new iris with:
- 7.2 cm sepal length, 2.5 cm sepal width,
- 5.1 cm petal length and 1.5 cm petal width.

Here are some rows from the training data:

| sepal length | sepal width | petal length | petal width | species        |
|--------------|-------------|--------------|-------------|----------------|
| 4.6          | 3.2         | 1.4          | 0.2         | Iris-setosa    |
| 6.2          | 2.8         | 4.8          | 1.8         | Iris-virginica |

## Euclidean Distance Example:
<div>
<img src="https://github.com/merriekay/S23-CS167-Notes/blob/main/images/day03_euclidean_ex.png?raw=1"/ width="500" height="200">
</div>

## 💬 Group Exercise:

Unknown Instance:
- 7.2 cm sepal length, 2.6 cm sepal width
- 5.1 cm petal length and 1.5 cm petal width.

If this was our entire set of training examples:

| sepal length | sepal width | petal length | petal width | species         |
|--------------|-------------|--------------|-------------|-----------------|
| 4.6          | 3.2         | 1.4          | 0.2         | Iris-setosa     |
| 6.2          | 2.8         | 4.8          | 1.8         | Iris-virginica  |
| 6.2          | 2.2         | 4.5          | 1.5         | Iris-versicolor |
| 6.3          | 2.7         | 4.9          | 1.8         | Iris-virginica  |

### What would the 1-NN version predict? 2-NN? 3-NN? 4-NN?

## Distance Functions
You can use any distance function you want. Other common distances include:

### Manhattan Distance:
### $d(x_i, x_j) = \sum_{r=1}^{n} | a_r(x_i) - a_r(x_j)|$

### Minkowski distance:
### $d(x_i, x_j) = (\sum_{r=1}^{n} | a_r(x_i) - a_r(x_j)|^p)^{1/p}$

### Or you can make your own:
- Online dating use distance scores to predict who you will swipe left/right on
- Baseball similarity matrix
- etc.

# Let's implement kNN in Python/Pandas:

Let's build a 5-nearest-neighbor Iris classifier from scratch--using our Pandas/Python skills:

To implement this 5NN, we need to do 3 things:
1. Calculate the distances from each of the rows to the new instance
2. Sort the data by these distances
3. Select the k closest training examples and use them to predict the most commonly occuring class of the closest neighbors.

## 1. Calculate the Distances:
Let's start by adding a new column to our `iris` dataframe that is the distance from each existing row to the new instance with:
- 5.1 petal length, 7.2 sepal length, 1.5 petal width, and 2.5 sepal width

The syntax for adding a new column is `df['new col name'] = ___________`

In [4]:
import numpy as np
new_iris = {}
new_iris['petal length'] = 5.1
new_iris['sepal length'] = 7.2
new_iris['petal width'] = 1.5
new_iris['sepal width'] = 2.5

In [5]:
#import the data:
import pandas as pd

path2 = '/content/drive/MyDrive/cs167_sp25/datasets/irisData.csv'
iris= pd.read_csv(path2)
iris.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [6]:
iris_dist     = (new_iris['petal length'] - iris['petal length'])**2 + \
                (new_iris['sepal length'] - iris['sepal length'])**2 + \
                (new_iris['petal width']  - iris['petal width'])**2 + \
                (new_iris['sepal width']  - iris['sepal width'])**2

# insert the distance into a new column in the same DataFrame, 'iris', as follows:
iris['distance_column'] = np.sqrt(iris_dist)
#iris['distance_column'] = iris_dist**0.5 # it also calculates the square root
iris.head()

,sepal length,sepal width,petal length,petal width,species,distance_column
0,5.1,3.5,1.4,0.2,Iris-setosa,4.559605
1,4.9,3.0,1.4,0.2,Iris-setosa,4.573839
2,4.7,3.2,1.3,0.2,Iris-setosa,4.782259
3,4.6,3.1,1.5,0.2,Iris-setosa,4.665833
4,5.0,3.6,1.4,0.2,Iris-setosa,4.629255


## 2. Sort the data by the distance
Let's now sort our data using the built in `sort_values()` function. [ [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) ]

We want to find the nearest k neighbors, so sorting them in ascending order (which is the default setting for `sort_values()` will work nicely.

In [9]:
iris_sorted = iris.sort_values(by='distance_column')
#iris_sorted = iris.sort_values(by='sepal length')
iris_sorted.head(8) #shortest distances first

,sepal length,sepal width,petal length,petal width,species,distance_column
76,6.8,2.8,4.8,1.4,Iris-versicolor,0.591608
52,6.9,3.1,4.9,1.5,Iris-versicolor,0.700000
77,6.7,3.0,5.0,1.7,Iris-versicolor,0.741620
50,7.0,3.2,4.7,1.4,Iris-versicolor,0.836660
129,7.2,3.0,5.8,1.6,Iris-virginica,0.866025
86,6.7,3.1,4.7,1.5,Iris-versicolor,0.877496
58,6.6,2.9,4.6,1.3,Iris-versicolor,0.900000
54,6.5,2.8,4.6,1.5,Iris-versicolor,0.911043


## 3. Display the most common species among these 5


In [21]:
k = 5
print(f'Sorted distances closest {k}')
top_k_samples = iris_sorted.iloc[0:k]['species']
#top_k_samples = iris_sorted.iloc[0:k][ ["sepal length", "species", "distance_column"] ]
top_k_samples.head(k)

Sorted distances closest 5


,species
76,Iris-versicolor
52,Iris-versicolor
77,Iris-versicolor
50,Iris-versicolor
129,Iris-virginica


In [22]:
predicted_label = top_k_samples.mode()
print(f'(Most frequent) label of the new sample according to {k}-NN: {predicted_label[0]}')

#predicted_sepal_length = top_k_samples['sepal length'].mode()
#print(f'(Most frequent) sepal length of the new sample according to {k}-NN: {predicted_sepal_length[0]}')

(Most frequent) label of the new sample according to 5-NN: Iris-versicolor


And Viola! We have successfully implemented our first machine learning model from scratch.

## 💬 Discussion Question

What part of the code would you change to use some other value of k in the kNN?

What would you change if you wanted to classify a different new Iris?

## Quick Function Refresher:

In [ ]:
# reminder of what a function looks like in Python:
def find_min(a_list):
    smallest = 10000000
    for element in a_list:
        if element < smallest:
            smallest=element
    return smallest

In [ ]:
find_min([2,5, -4, -54,14, 30])

-54

# Group Programming Exercise:

Rewrite our k-NN code so that it's a function.

Pass the iris measurements (specimen), dataframe, and k as parameters and return the predicted class.

Here's some starter code:

In [ ]:
def kNN(new_iris, iris, k):
    # write your code in here to make this function work
    predicted_label = -1

    # 1. calculate distances
    # your code here
    # ...


    # 2. sort
    # your code here
    # ...


    # 3. predict
    # your code here
    # ...


    return predicted_label

In [ ]:
new_iris = {}
new_iris['petal length'] = 5.1
new_iris['sepal length'] = 7.2
new_iris['petal width'] = 1.5
new_iris['sepal width'] = 2.5

# call the function you just wrote
predicted_label = kNN(new_iris, iris, 55)
print(f'Label of the new sample according to {k}-NN: {predicted_label[0]}')


Label of the new sample according to 5-NN: Iris-virginica
